# Fake News Classifier 


### Fake News Classifier to classify whether a given news sample is reliable or unreliable by using Bi-directional LSTM

## Dataset 

train.csv: A full training dataset with the following attributes:

id: unique id for a news article

title: the title of a news article

author: author of the news article

text: the text of the article; could be incomplete

label: a label that marks the article as potentially unreliable

1: unreliable
0: reliable

test.csv: A testing training dataset with all the same attributes at 
train.csv without the label.
##### link: https://www.kaggle.com/competitions/fake-news/data

In [87]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

import nltk
import re
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

from nltk.stem.porter import PorterStemmer
from sklearn.metrics import confusion_matrix,accuracy_score



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
df=pd.read_csv('/content/train.csv',engine='python',error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 13535: field larger than field limit (131072)
Skipping line 14005: unexpected end of data
Skipping line 13541: Expected 5 fields in line 13541, saw 6


In [31]:
df.shape

(14001, 5)

In [32]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14001 entries, 0 to 14000
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      14001 non-null  object 
 1   title   13623 non-null  object 
 2   author  12659 non-null  object 
 3   text    13955 non-null  object 
 4   label   13985 non-null  float64
dtypes: float64(1), object(4)
memory usage: 547.0+ KB


In [34]:
df.isnull().sum()

id           0
title      378
author    1342
text        46
label       16
dtype: int64

In [35]:
df=df.dropna()

In [36]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [40]:
X=df[['id',	'title','author', 'text']]   # independent features
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [41]:
y=df['label']

In [42]:
y.value_counts()

0.0    6976
1.0    5307
Name: label, dtype: int64

In [46]:
voc_size=5000    # vocabulary size

In [56]:
message_data=X.copy()    #copy data 

In [57]:
message_data.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [58]:
message_data.reset_index(inplace=True)   #index reset

## Dataset Preprocessing

In [88]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(message_data)):
    content = re.sub('[^a-zA-Z]', ' ', message_data['title'][i])  ## remove Anything except a..z and A..Z
    content = content.lower() # lower the data
    content = word_tokenize(content)  #word token
    
    content = [ps.stem(word) for word in content if not word in stopwords.words('english')] #remove stopword
    content = ' '.join(content)
    corpus.append(content)

In [68]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [69]:
#one hot encoding
onehot_data=[one_hot(words,voc_size) for words in corpus]
onehot_data

[[3642, 2743, 2662, 4905, 1647, 3283, 1378, 2072, 4077, 2434],
 [612, 734, 2314, 4684, 4562, 1140, 1338],
 [3260, 4432, 3304, 1161],
 [531, 3306, 3253, 2935, 715, 2407],
 [3339, 4562, 3450, 2624, 2223, 2445, 4562, 2126, 2083, 43],
 [4458,
  3914,
  1471,
  1430,
  3686,
  1272,
  3989,
  3297,
  1425,
  3008,
  857,
  12,
  2914,
  4784,
  1338],
 [677, 3807, 2731, 1361, 2235, 1093, 3170, 4514, 4998, 1732, 3962],
 [2438, 2473, 4345, 1440, 1531, 391, 1272, 2275, 4998, 1732, 3962],
 [347, 2459, 3941, 3416, 2998, 4847, 2057, 1253, 1272, 899],
 [835, 711, 2440, 663, 3661, 3882, 1717, 3571],
 [4953, 2848, 4536, 4712, 1110, 4781, 2217, 3882, 2733, 3151, 4889],
 [2935, 764, 1647, 4847, 1272, 1531],
 [1832, 3030, 571, 425, 2786, 697, 3285, 73, 4838],
 [3914, 1060, 151, 4720, 4755, 4458, 4780, 4998, 1732, 3962],
 [2777, 2992, 3197, 1822, 901, 4998, 1732, 3962],
 [2942, 4850, 726, 2705, 58, 3597, 4816, 3290, 3406, 2703],
 [3929, 1832, 734],
 [2310, 4872, 1762, 4555, 1272, 1840, 3660, 1338],
 [36

## Embedding

In [71]:
sent_length=20
embedded_data=pad_sequences(onehot_data,padding='pre',maxlen=sent_length)

embedded_data


array([[   0,    0,    0, ..., 2072, 4077, 2434],
       [   0,    0,    0, ..., 4562, 1140, 1338],
       [   0,    0,    0, ..., 4432, 3304, 1161],
       ...,
       [   0,    0,    0, ...,  813, 3293, 1338],
       [   0,    0,    0, ..., 4998, 1732, 3962],
       [   0,    0,    0, ..., 4210, 2877, 1338]], dtype=int32)

## model

In [73]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))   # Bidirectional(LSTM layer
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [77]:
X_data=np.array(embedded_data)
y_data=np.array(y)

### train test split

In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=42)

### Model Training

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=32)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred=np.where(y_pred>=0.5,1,0)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
print(accuracy_score(y_test,y_pred))